In [17]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

In [28]:

def make_request(uri, max_retries = 5):
    current_try = 1
    def fire_away(uri,current_try):
        if current_try < max_retries:
            try:
                response = requests.get(uri)
                if response.status_code == 200:
                    return json.loads(response.content)
            except:
                current_try+=1
                time.sleep(1)
                fire_away(uri,current_try)
        else:
            print('Cannot establish connection or wrong uri',uri)
            return None
    return fire_away(uri,current_try)

#make_request('https://api.pushshift.ioo/reddit/search/submission/?q=science&size=2&fields=selftext,title,author,url,id,full_link')    

Cannot establish connection or wrong uri https://api.pushshift.ioo/reddit/search/submission/?q=science&size=2&fields=selftext,title,author,url,id,full_link


In [29]:
end_at = math.ceil(datetime.utcnow().timestamp())
print(end_at)

1550494635
